<a href="https://colab.research.google.com/github/SunnyORZ030/CMPE255-Modern-AI-with-unsloth.ai/blob/main/Preference_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 先移除會卡版本的舊套件（不存在也沒關係）
!pip uninstall -y transformers trl peft accelerate datasets pyarrow fsspec gcsfs || true
!pip uninstall -y cudf-cu12 pylibcudf-cu12 dask-cudf-cu12 cuml-cu12 cugraph-cu12 bigframes || true

# 可選：清除 pip 快取，避免抓到舊輪子
!pip cache purge

Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: peft 0.17.1
Uninstalling peft-0.17.1:
  Successfully uninstalled peft-0.17.1
Found existing installation: accelerate 1.11.0
Uninstalling accelerate-1.11.0:
  Successfully uninstalled accelerate-1.11.0
Found existing installation: datasets 4.0.0
Uninstalling datasets-4.0.0:
  Successfully uninstalled datasets-4.0.0
Found existing installation: pyarrow 21.0.0
Uninstalling pyarrow-21.0.0:
  Successfully uninstalled pyarrow-21.0.0
Found existing installation: fsspec 2024.5.0
Uninstalling fsspec-2024.5.0:
  Successfully uninstalled fsspec-2024.5.0
Found existing installation: gcsfs 2024.5.0
Uninstalling gcsfs-2024.5.0:
  Successfully uninstalled gcsfs-2024.5.0
Files removed: 48 (45.8 MB)


In [4]:
!pip install -U \
  "pyarrow==21.0.0" "fsspec==2024.5.0" "gcsfs==2024.5.0" \
  "transformers==4.56.2" "trl==0.23.0" "peft==0.12.0" "accelerate==1.4.0" "datasets==4.3.0"

  Using cached pyarrow-21.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached fsspec-2024.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached gcsfs-2024.5.0-py2.py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 58.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 168.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 26.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8/8 [gcsfs]


In [5]:
import transformers, datasets, peft, accelerate, importlib
from trl import DPOTrainer

print("transformers:", transformers.__version__)  # 期望 4.57.2（或 4.56.2）
print("trl:", __import__("trl").__version__)      # 期望 0.23.0
print("peft:", peft.__version__)                  # 期望 0.13.x（或 0.12.x）
print("accelerate:", accelerate.__version__)      # 期望 1.4.0
print("datasets:", datasets.__version__)          # 期望 4.3.0
print("✅ DPOTrainer 可匯入")

transformers: 4.56.2
trl: 0.23.0
peft: 0.12.0
accelerate: 1.4.0
datasets: 4.3.0
✅ DPOTrainer 可匯入


In [6]:
import os, json
os.makedirs("/content/data", exist_ok=True)

rows = [
  {
    "prompt": "Explain cross-validation in one sentence.",
    "chosen": "Cross-validation splits data into folds to estimate generalization reliably.",
    "rejected": "Cross-validation makes the model overfit less by training on the test set."
  },
  {
    "prompt": "Give a short tip to avoid overfitting.",
    "chosen": "Use regularization and early stopping, and validate on held-out data.",
    "rejected": "Always train longer with higher learning rate."
  },
  {
    "prompt": "Difference between classification and regression?",
    "chosen": "Classification predicts discrete labels, while regression predicts continuous values.",
    "rejected": "Both predict numbers only."
  },
  {
    "prompt": "What is overfitting?",
    "chosen": "When a model memorizes training noise and performs poorly on new data.",
    "rejected": "When a model trains too fast and accuracy is always zero."
  },
]

with open("/content/data/pref_dataset.jsonl","w") as f:
    for r in rows:
        f.write(json.dumps(r, ensure_ascii=False) + "\n")

# 快速檢查
!head -n 3 /content/data/pref_dataset.jsonl

{"prompt": "Explain cross-validation in one sentence.", "chosen": "Cross-validation splits data into folds to estimate generalization reliably.", "rejected": "Cross-validation makes the model overfit less by training on the test set."}
{"prompt": "Give a short tip to avoid overfitting.", "chosen": "Use regularization and early stopping, and validate on held-out data.", "rejected": "Always train longer with higher learning rate."}
{"prompt": "Difference between classification and regression?", "chosen": "Classification predicts discrete labels, while regression predicts continuous values.", "rejected": "Both predict numbers only."}


In [7]:
from datasets import load_dataset

ds = load_dataset("json", data_files={"train": "/content/data/pref_dataset.jsonl"})
train_ds = ds["train"]
train_ds[0]

Generating train split: 0 examples [00:00, ? examples/s]

{'prompt': 'Explain cross-validation in one sentence.',
 'chosen': 'Cross-validation splits data into folds to estimate generalization reliably.',
 'rejected': 'Cross-validation makes the model overfit less by training on the test set.'}

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_NAME = "HuggingFaceTB/SmolLM2-135M-Instruct"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float32,          # CPU 用 FP32 最穩
    device_map={"": "cpu"},
    trust_remote_code=True,
)

model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

print("Loaded!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

config.json:   0%|          | 0.00/861 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

Loaded!


In [9]:
INSTR = "You are a helpful assistant. Answer briefly."

def format_prompt(p):
    return f"### Instruction:\n{INSTR}\n\n### Input:\n{p}\n\n### Response:\n"

def map_to_dpo(batch):
    # DPOTrainer 允許我們直接傳 raw prompt / chosen / rejected
    return {
        "prompt": [format_prompt(p) for p in batch["prompt"]],
        "chosen": batch["chosen"],
        "rejected": batch["rejected"],
    }

train_dpo = train_ds.map(map_to_dpo, batched=True, remove_columns=train_ds.column_names)
train_dpo[0]

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

{'prompt': '### Instruction:\nYou are a helpful assistant. Answer briefly.\n\n### Input:\nExplain cross-validation in one sentence.\n\n### Response:\n',
 'chosen': 'Cross-validation splits data into folds to estimate generalization reliably.',
 'rejected': 'Cross-validation makes the model overfit less by training on the test set.'}

In [10]:
from peft import LoraConfig, get_peft_model, TaskType

lora_cfg = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4, lora_alpha=16, lora_dropout=0.1,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none"
)

model = get_peft_model(model, lora_cfg)
model.print_trainable_parameters()

trainable params: 1,221,120 || all params: 135,736,128 || trainable%: 0.8996


In [15]:
# === Step 6：用 DPOConfig 啟動 DPOTrainer（CPU 相容，TRL 0.23.0）===
import trl
from trl import DPOTrainer, DPOConfig
from types import SimpleNamespace

print("TRL version:", trl.__version__)

MAX_LEN = 256

# 1) 建立 DPOConfig：關閉 bf16/fp16，補上必要欄位
dpo_args = DPOConfig(
    # ---- 基本訓練參數 ----
    output_dir="/content/dpo_lora_cpu_out",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    max_steps=150,
    learning_rate=5e-5,
    warmup_ratio=0.05,
    weight_decay=0.0,
    logging_steps=10,
    save_steps=75,
    save_total_limit=1,
    report_to=[],

    # ---- 在 CPU 上一定要關閉的混合精度 / GPU 設定 ----
    bf16=False,
    fp16=False,
    bf16_full_eval=False,
    fp16_full_eval=False,

    # ---- 其他穩定性設定（CPU 友善）----
    optim="adamw_torch",
    remove_unused_columns=False,
    dataloader_pin_memory=False,

    # ---- DPOTrainer 會讀取的長度與 padding 值 ----
    max_length=MAX_LEN,
    max_prompt_length=MAX_LEN // 2,
    padding_value=int(tokenizer.pad_token_id),

    # ---- 這個版本會讀取的欄位（避免 AttributeError）----
    model_init_kwargs={},
)

# 2) 啟動 DPOTrainer：用 processing_class=tokenizer（這版不吃 tokenizer=...）
trainer = DPOTrainer(
    model=model,
    ref_model=None,             # 讓 DPOTrainer 內部自建 reference model（不訓練）
    args=dpo_args,
    train_dataset=train_dpo,    # 必須包含 prompt / chosen / rejected
    processing_class=tokenizer, # 這版的介面用 processing_class 來傳 tokenizer
)

trainer.train()

TRL version: 0.23.0


Extracting prompt in train dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/4 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,0.650600
20,0.424000
30,0.222800
40,0.104400
50,0.048000
60,0.024400
70,0.014400
80,0.009800
90,0.007400
100,0.006100


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


TrainOutput(global_step=150, training_loss=0.10226585658888022, metrics={'train_runtime': 1509.7064, 'train_samples_per_second': 0.795, 'train_steps_per_second': 0.099, 'total_flos': 0.0, 'train_loss': 0.10226585658888022, 'epoch': 150.0})

In [16]:
save_dir = "/content/dpo_lora_smolm2_cpu"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)

Saved to: /content/dpo_lora_smolm2_cpu


In [17]:
import torch
model.eval()

def chat(prompt, max_new_tokens=64):
    text = format_prompt(prompt)
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        out = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,                 # 先關抽樣，避免 NaN 機率問題
            temperature=None,
            top_p=None,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    decoded = tokenizer.decode(out[0], skip_special_tokens=True)
    return decoded.split("### Response:")[-1].strip()

print(chat("Explain cross-validation in one sentence."))
print("----")
print(chat("Give a short tip to avoid overfitting."))

Cross-validation is a statistical technique used to evaluate the performance of a model on multiple subsets of data, allowing for the estimation of the model's performance on unseen data.
----
To avoid overfitting, it's essential to avoid using the same model for different subsets of your data. This helps prevent the model from fitting the noise in the training data and generalizes poorly to new, unseen data.


In [18]:
!cd /content && zip -r dpo_lora_smolm2_cpu.zip dpo_lora_smolm2_cpu dpo_lora_cpu_out data
print("Zip ready at /content/dpo_lora_smolm2_cpu.zip")

  adding: dpo_lora_smolm2_cpu/ (stored 0%)
  adding: dpo_lora_smolm2_cpu/merges.txt (deflated 55%)
  adding: dpo_lora_smolm2_cpu/README.md (deflated 66%)
  adding: dpo_lora_smolm2_cpu/tokenizer_config.json (deflated 87%)
  adding: dpo_lora_smolm2_cpu/vocab.json (deflated 59%)
  adding: dpo_lora_smolm2_cpu/adapter_config.json (deflated 53%)
  adding: dpo_lora_smolm2_cpu/tokenizer.json (deflated 82%)
  adding: dpo_lora_smolm2_cpu/chat_template.jinja (deflated 42%)
  adding: dpo_lora_smolm2_cpu/adapter_model.safetensors (deflated 8%)
  adding: dpo_lora_smolm2_cpu/special_tokens_map.json (deflated 76%)
  adding: dpo_lora_cpu_out/ (stored 0%)
  adding: dpo_lora_cpu_out/checkpoint-150/ (stored 0%)
  adding: dpo_lora_cpu_out/checkpoint-150/merges.txt (deflated 55%)
  adding: dpo_lora_cpu_out/checkpoint-150/training_args.bin (deflated 53%)
  adding: dpo_lora_cpu_out/checkpoint-150/README.md (deflated 66%)
  adding: dpo_lora_cpu_out/checkpoint-150/optimizer.pt (deflated 7%)
  adding: dpo_lora_c